In [ ]:
!pip show tensorflow

In [ ]:
!pip install git+https://github.com/tensorflow/examples.git


In [ ]:
import re
import cv2
import math
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from kaggle_datasets import KaggleDatasets
from tensorflow_examples.models.pix2pix import pix2pix
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras import optimizers
from tensorflow.keras import initializers, layers
from kaggle_datasets import KaggleDatasets
import pickle
import os
import glob
import matplotlib.pyplot as plt

# f= open("guru99.txt","w+")
# for i in range(10):
#      f.write("This is line %d\r\n" % (i+1))

print("start")

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()
    
print("REPLICAS: ", strategy.num_replicas_in_sync)




In [ ]:
SEED = 42
SIZE = [128,128]
# 
INPUT_SHAPE = (128, 128, 3)

np.set_printoptions(threshold=np.inf)

In [ ]:
def seed_everything(SEED):
    np.random.seed(SEED)
    tf.random.set_seed(SEED)

seed_everything(SEED)


In [ ]:

def _Float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def write_melanoma2020_input_tfrecords_file( out_path, images,image_names, labels=None) -> None:
    if labels != None:
        assert len(images) == len(labels)
    assert len(images) == len(image_names)
#     options=tf.io.TFRecordOptions(compression_type="GZIP",compression_level=3,mem_level=3)
    with tf.io.TFRecordWriter(out_path) as writer:
            for i in range(len(images)):
                img = tf.cast(images[i]*255, tf.uint8)
                img_bytes = tf.image.encode_png(img).numpy()
                image_name = image_names[i]
                if labels!=None:
                    label = labels[i]
                    data = {'image': _bytes_feature(img_bytes), 'target': _int64_feature(label),'image_name': _bytes_feature(image_name)}
                else:
                    data = {'image': _bytes_feature(img_bytes), 'image_name': _bytes_feature(image_name)}   
                feature = tf.train.Features(feature=data) 
                example = tf.train.Example(features=feature) 
                serialized = example.SerializeToString() 
                writer.write(serialized)  


In [ ]:
def decode_image(image):
    image = tf.image.decode_jpeg(image, channels=3) 
    image = tf.cast(image, tf.float32)/255.0
    image = tf.reshape(image, [*SIZE, 3])
    return image

def parse_example_melanoma(example,labeled):  
    if labeled:
       LABELED_TFREC_FORMAT ={
         "image": tf.io.FixedLenFeature([], tf.string),
         "target": tf.io.FixedLenFeature([], tf.int64),
         "image_name": tf.io.FixedLenFeature([], tf.string) }
    else:
       LABELED_TFREC_FORMAT ={
         "image": tf.io.FixedLenFeature([], tf.string),
         "image_name": tf.io.FixedLenFeature([], tf.string) } 
    parsed_example = tf.io.parse_single_example(example,LABELED_TFREC_FORMAT)
    image = decode_image(parsed_example['image'])
    image_name = parsed_example['image_name']
    if labeled:
        label = tf.cast(parsed_example['target'], tf.int64)
        return (image,image_name,label)
    else:
        return (image,image_name)

def load_dataset_melanoma(filenames,labeled=False, ordered=True):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False
    parse_function = lambda x: parse_example_melanoma(example=x,labeled=labeled)
    return (tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) 
              .with_options(ignore_order)
               .map(parse_function, num_parallel_calls=AUTO))

In [ ]:
def plot_images(np_images, titles = [], columns = 3, figure_size = (24, 15)):
    count = len(np_images)
    rows = math.ceil(count / columns)

    fig = plt.figure(figsize=figure_size)
    subplots = []
    for index in range(count):
        subplots.append(fig.add_subplot(rows, columns, index + 1))
        if len(titles):
            subplots[-1].set_title(str(titles[index]))
        plt.imshow(np_images[index])

    plt.show()

In [ ]:
def get_segmentation_unet_model():
    base_model = tf.keras.applications.MobileNetV2(
        input_shape=[INPUT_SHAPE[0], INPUT_SHAPE[1], INPUT_SHAPE[2]], include_top=False)

    # Use the activations of these layers
    layer_names = [
        'block_1_expand_relu',  # 64x64
        'block_3_expand_relu',  # 32x32
        'block_6_expand_relu',  # 16x16
        'block_13_expand_relu',  # 8x8
        'block_16_project',  # 4x4
    ]
    layers = [base_model.get_layer(name).output for name in layer_names]

    # Create the feature extraction model
    down_stack = tf.keras.Model(inputs=base_model.input, outputs=layers)

    down_stack.trainable = False
    up_stack = [
        pix2pix.upsample(512, 3),  # 4x4 -> 8x8
        pix2pix.upsample(256, 3),  # 8x8 -> 16x16
        pix2pix.upsample(128, 3),  # 16x16 -> 32x32
        pix2pix.upsample(64, 3),  # 32x32 -> 64x64
    ]

    def unet_model(output_channels):
        inputs = tf.keras.layers.Input(shape=[INPUT_SHAPE[0],INPUT_SHAPE[1], INPUT_SHAPE[2]])
        x = inputs

        # Downsampling through the model
        skips = down_stack(x)
        x = skips[-1]
        skips = reversed(skips[:-1])

        # Upsampling and establishing the skip connections
        for up, skip in zip(up_stack, skips):
            x = up(x)
            concat = tf.keras.layers.Concatenate()
            x = concat([x, skip])

        # This is the last layer of the model
        last = tf.keras.layers.Conv2DTranspose(
            output_channels, 3, strides=2,
            padding='same')  # 64x64 -> 128x128

        x = last(x)

        return tf.keras.Model(inputs=inputs, outputs=x)

    model = unet_model(INPUT_SHAPE[2])
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model

segmentation_model = get_segmentation_unet_model()


In [ ]:
def create_final_image(main_images, predict_images):
    predict_images=np.logical_not(predict_images)
#     print("result:",predict_images.shape)

# Return images that are not segmented well enough
    zero_counts = np.count_nonzero(predict_images,(1,2))
#     print("Zeros",zero_counts.shape,zero_counts)
    for i in range(0,zero_counts.shape[0]):
        if zero_counts[i]<400 :
#             print("Wrng ",i,zero_counts[i])
#             plot_images(np.reshape(main_images[i],(1,128,128,3)))
#             plot_images(np.reshape(predict_images[i],(1,128,128)))
            predict_images[i]=np.ones((predict_images.shape[1],predict_images.shape[2]))
    
    reshaped_masks = np.reshape(predict_images,(main_images.shape[0],main_images.shape[1],main_images.shape[2],1))
    results = main_images * np.tile(reshaped_masks,(1,1,3))
    return results

def segment_image(input_imgs):
    pred_masks= segmentation_model.predict(input_imgs)
    pred_masks = tf.argmax(pred_masks, axis=-1)
    return create_final_image(input_imgs,pred_masks)

In [ ]:
DATASET = 'melanoma-128x128'
GCS_PATH = KaggleDatasets().get_gcs_path(DATASET)
train_filenames = tf.io.gfile.glob(GCS_PATH + '/train*.tfrec')
test_filenames = tf.io.gfile.glob(GCS_PATH + '/test*.tfrec')
DATASET = 'isic2019-128x128'
GCS_PATH = KaggleDatasets().get_gcs_path(DATASET)
train2019_filenames = tf.io.gfile.glob(GCS_PATH + '/train*.tfrec')

segmentation_model.load_weights("/kaggle/input/unet-segment-weights/weights")

def segment_save(filenames, tf_record_name,labeled=True):
    segment_batch_size = 128
    index = 0
    tf_record_count = len(filenames)
    for tf_record_number in range(0, tf_record_count):
        print("tf_record_number" ,tf_record_number, "from", tf_record_count - 1)
        dataset = load_dataset_melanoma(filenames[tf_record_number],labeled=labeled).batch(segment_batch_size)
        segmented_images = []
        segmented_lables = []
        segmented_image_names =[]
        batch_number =0
        for item in dataset:
            if labeled:
                (images,image_names,labels)=item
                labels=labels.numpy()
            else:
                (images,image_names)=item
            image_names = image_names.numpy()
            batch_number +=1 
            segment_result = segment_image(images)
            segment_result = images
            segment_result_array = np.split(segment_result, images.shape[0])
            for image_number in range(0, images.shape[0]):
                segmented_images.append(np.squeeze(segment_result_array[image_number], 0))
                if labeled:
                    segmented_lables.append(labels[image_number])
                segmented_image_names.append(image_names[image_number])
        save_path = tf_record_name + str(index).zfill(2) + "-" + str(len(segmented_images)) + ".tfrec"
        if labeled:
            write_melanoma2020_input_tfrecords_file(save_path, segmented_images,segmented_image_names, segmented_lables)
        else:
            write_melanoma2020_input_tfrecords_file(save_path, segmented_images,segmented_image_names)
        index += 1
        

segment_save(train_filenames, "train_2020_")
segment_save(train2019_filenames, "train_2019_")
segment_save(test_filenames, "test_",False)

In [ ]:
# !rm  /kaggle/working/data_2020.tar.gz
# !tar -zcvf data_2020.tar.gz train_2020*
# !rm /kaggle/working/train_2020*.tfrec

# !rm  /kaggle/working/data_2019.tar.gz
# !tar -zcvf data_2019.tar.gz train_2019*
# !rm /kaggle/working/train_2019*.tfrec

# !rm  /kaggle/working/data_test_2020.tar.gz
# !tar -zcvf data_test_2020.tar.gz test*
# !rm /kaggle/working/data_test_2020*.tfrec





<a href="./data_2019.tar.gz"> Download File 2019 Train</a>

<a href="./data_2020.tar.gz"> Download File 2020 Train</a>

<a href="./data_test_2020.tar.gz"> Download File 2020 Test</a>


